# Notebook 5: CLIP and Multimodal Self-Supervision

Self-supervised learning isn’t limited to single modalities. An exciting development is multimodal self-supervision, where models learn from the natural pairing of data across modalities (e.g., images and text). The prime example is CLIP (Contrastive Language–Image Pre-training) by OpenAI, which learns joint representations of images and text. This notebook explores how CLIP works and demonstrates its powerful capabilities.

## CLIP Overview

CLIP consists of two neural networks:
- **Image Encoder:** e.g., a ResNet-50 or Vision Transformer that turns an image into an embedding vector.
- **Text Encoder:** e.g., a Transformer (like a smaller GPT) that turns an input text (like a caption or label) into an embedding vector.

During training, CLIP is fed a batch of (image, text) pairs that naturally co-occur (like an image and its caption on the internet). It then maximizes the similarity of the embeddings for each true image-text pair, while minimizing it for mismatched pairs. Concretely, a contrastive loss is used: each image should be most similar to its corresponding text and not to others in the batch, and vice versa for each text. This is usually implemented via a cross-entropy loss where the correct image-text pair has the highest similarity score among all pairings in the batch.

**Dual-Encoder Architecture:** The image and text encoders process their inputs independently, and interactions happen only through the similarity computation in the loss. This means after training, we can embed any image and any text, and compare them in the shared embedding space.

What does this achieve? It makes the embedding space multimodal and semantically meaningful:
- Similar images will be near each other.
- Similar texts will be near each other.
- Each image will also be near texts that describe its content, and each text near images illustrating it.

**Scale:** CLIP was trained on a very large dataset (around 400 million image-text pairs from the web). This broad data exposes it to a huge variety of content (from dogs and cats to abstract art and infographics). Models trained on such data exhibit impressive generalization.

## Zero-Shot Image Classification with CLIP

One of CLIP’s stunning abilities is zero-shot classification. Because CLIP learned to connect images with text, you can classify an image by simply providing the names of the categories in text form, without any further training on that specific task. For example, suppose you want to classify images into {"cat", "dog", "horse"}. With CLIP, you can do:

- For each category, write a prompt like "a photo of a cat", "a photo of a dog", "a photo of a horse." (Engineering the right prompt can slightly affect accuracy; common choices include "a photo of a {class}" or just the class name.)
- Use the CLIP text encoder to get an embedding for each prompt.
- Use the CLIP image encoder to get an embedding for the query image.
- Compute similarity (e.g., cosine similarity) between the image embedding and each text embedding.
- Pick the highest similarity – that text’s label is the prediction.

Because CLIP has learned to put, say, actual cat images near the text "a photo of a cat" in embedding space, this procedure works remarkably well.

### Example Code: Using HuggingFace’s CLIP model for zero-shot prediction.

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

# Load CLIP pre-trained model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Prepare candidate class prompts
labels = ["a photo of a cat", "a photo of a dog", "a photo of a horse"]

# Load an image (let’s say an image of a dog)
url = "https://images.unsplash.com/photo-1603312744-45caa8125824"  # dog image
image = Image.open(requests.get(url, stream=True).raw).resize((224,224))

# Preprocess image and text through CLIPProcessor
inputs = processor(text=labels, images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)

logits_per_image = outputs.logits_per_image  # shape [1, num_labels]
probs = logits_per_image.softmax(dim=1)      # softmax to get probabilities
for label, prob in zip(labels, probs[0].tolist()):
    print(f"{label}: {prob:.4f}")

# end of block

It correctly classifies the image as a dog with very high confidence, without any task-specific training! This zero-shot ability extends to hundreds or even thousands of categories, as long as you can describe them. The CLIP paper demonstrated zero-shot classification on ImageNet (1000 classes) by just using the class names in prompts, achieving around 76% top-1 accuracy (which is on par with a supervised ResNet-50). This was a big deal: it means the model learned enough about those 1000 object types from web data without ever explicitly being trained on ImageNet labels.

## Beyond Classification: CLIP for Retrieval and Search

Since CLIP has a joint image-text space, you can use it for:
- **Image search by text (Text-to-Image Retrieval):** Provide a sentence, find the image whose embedding is closest. For instance, search your image database for "a red sports car on a mountain road" – CLIP can rank images by how well they match this description. This is like a Google image search without explicit tags, using the model’s semantic knowledge.
- **Image-to-Text Retrieval:** The opposite – given an image, find which caption or description (from a set) fits best. This can be used in captioning systems (retrieve the nearest caption from a large pool), or more simply, to label images if you have a set of candidate labels.
- **Finding similarities:** You could cluster images by their CLIP embeddings to see groups of similar content, or cluster text and see which concepts are close.

### Practical example – Image Retrieval

Imagine we have a small gallery of images and we want to find which one best matches a query text. We can do:

In [ ]:
# Assume we have a list of image URLs or PIL images in `image_list`

image_list = [Image.open(requests.get(u, stream=True).raw).resize((224,224)) for u in [url1, url2, url3, url4]]
query = "a busy city street at night"

# Preprocess images and text
image_inputs = processor(images=image_list, return_tensors="pt", padding=True)
text_inputs = processor(text=[query], return_tensors="pt", padding=True)

# Compute features
with torch.no_grad():
    image_features = model.get_image_features(**image_inputs)
    text_features = model.get_text_features(**text_inputs)
# Normalize features
image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)
# Cosine similarities of the text query to each image:
sims = (image_features @ text_features.T).squeeze()  # shape [len(image_list)]
best_idx = sims.argmax().item()
print("Most matching image index:", best_idx)

# end python block

This will tell us which image in `image_list` best matches "a busy city street at night". If our images included, say, a city skyline, a forest, a highway, and a night city street, it should pick the night city street image. We can likewise do image-to-text: if we had a list of possible captions, we’d embed them and find which is closest to the image embedding.

## Fine-Tuning CLIP for Specific Tasks

While zero-shot is great, sometimes you want to improve performance on a narrower task. CLIP’s embeddings can be fine-tuned. Options include:
- Fine-tune the entire CLIP model on your (image, text) data if you have some labeled pairs.
- Or more commonly, use CLIP’s image encoder as a feature extractor for a downstream vision task. For example, take CLIP’s image embeddings and train a new classifier on top of them with your small dataset. This usually outperforms training a CNN from scratch on small data.

### Example – Fine-tuning CLIP on CIFAR-10

(CIFAR-10 has 10 classes like airplane, dog, truck, etc.) Even though CLIP can do zero-shot on CIFAR by prompts, let's say we want the best accuracy. We could take the CLIP image encoder and add a new linear layer for 10 classes and train it using CIFAR-10 labeled data (likely with a lower learning rate for the pre-trained backbone, to not distort it too much). This typically yields very high accuracy quickly, because CLIP’s features are already very discriminative for these classes. However, fine-tuning should be done cautiously – CLIP’s text alignments can also get altered if you fine-tune just on one modality or a small dataset.

**Compute Consideration:** CLIP pre-training is heavy (requiring lots of data and GPU). But using CLIP is lightweight – a forward pass through the image and text encoder is all. That’s why CLIP is popular in many applications; one can leverage the pre-trained model via libraries like transformers without retraining it.

## CLIP Limitations and Bias

It’s worth noting:
- **Bias and Fairness:** CLIP inherits biases from web data. It might underperform or misclassify certain demographics or objects in biased ways (OpenAI noted some issues in the paper). Always be mindful of this if deploying CLIP.
- **Robustness:** CLIP is surprisingly robust to some distribution shifts (it knows a lot of styles, sketches, etc.), but it can also be fooled by adversarial cues (like adding text in an image can throw it off, because it also reads text in images).
- **Prompt Engineering:** The performance can depend on the prompt phrasing. People have found that adding certain prefixes (like "a photo of a") helps more than just the raw class name. There's a mini-field of "prompt engineering" or even learning prompts (so-called prompt tuning). In our notebooks, we might not dive deep into that, but it’s something practitioners do to squeeze more accuracy out of CLIP for zero-shot tasks.

## Hands-On: Using CLIP for Image & Text Retrieval

Let's do a small end-to-end exercise with CLIP:
- Use CLIP to encode a set of images.
- Use CLIP to encode a set of text descriptions.
- Compute a similarity matrix and find best matches.

Say we have three images: a car, a dog, and a painting of sunset. And three texts: "a dog running on grass", "a sunset over the mountains", "a red sports car on a track". We expect the dog text to match the dog image, etc.

In [ ]:
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch

# Let's load 3 images from URLs (for demonstration, pre-selected URLs)
urls = [
    "https://images.unsplash.com/photo-1516483638261-f4dbaf036963",  # a dog
    "https://images.unsplash.com/photo-1504215680853-026ed2a45def",  # a red car
    "https://images.unsplash.com/photo-1502086223501-7ea6ecd79368"   # a sunset
]
images = [Image.open(requests.get(u, stream=True).raw).convert("RGB") for u in urls]
# Preprocess images (CLIPProcessor can do this, but let's manually to illustrate normalization)
preprocess = Compose([
    Resize(224, interpolation=Image.BICUBIC),
    CenterCrop(224),
    ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),  # CLIP's normalization
])
image_tensors = torch.stack([preprocess(img) for img in images])
texts = ["a dog running on grass", "a photo of a red sports car on a track", "a beautiful sunset over the mountains"]

# Compute features
with torch.no_grad():
    img_features = model.get_image_features(image_tensors.to(model.device))
    text_features = model.get_text_features(**processor(text=texts, return_tensors="pt", padding=True).to(model.device))
# Normalize
img_features = img_features / img_features.norm(dim=1, keepdim=True)
text_features = text_features / text_features.norm(dim=1, keepdim=True)
# Similarity matrix
sim_matrix = img_features @ text_features.T  # shape [3,3]
sim_matrix = sim_matrix.cpu().numpy()
print("Similarity matrix:\n", sim_matrix)

## Fine-Tuning CLIP (Bonus)

If time permits, one might fine-tune CLIP’s image encoder for a domain-specific task. For instance, perhaps we have a medical imaging dataset with text reports. We could initialize a CLIP model and fine-tune on those pairs to adapt the embeddings to the medical domain. That would be similar to how BioBERT or other domain-specific BERTs are made (they start with BERT then continue pre-training on domain text). However, fine-tuning both image and text encoders on small data can lead to overfitting; often one encoder is kept fixed, or a lightweight adaptation is done (like learning just a linear combination of the outputs).

## Wrap Up

CLIP demonstrates the synergy of combining modalities in self-supervision:
- **Leverage free supervision:** The web is full of image-text pairs (every meme, every product photo with description, etc., is training data).
- **Generalize broadly:** By learning from language, the model inherits some of the generality of language. You can specify novel categories or attributes at inference time via text that the model may never have seen labeled, yet it can recognize them if they were described in the training data. This “open vocabulary” trait is extremely powerful.
- **Multimodal understanding:** CLIP (and successors like ALIGN, FILIP, etc.) kind of “understand” images in a way we do – by relating them to words. This is why you can ask CLIP to do tasks in plain English.

In Notebook 6 (the final one), we will discuss some advanced applications building on these representations, including how to utilize models like CLIP for things like few-shot learning, image retrieval systems with vector databases, and even connecting CLIP with generative models. We’ll also mention other multimodal SSL developments such as aligning audio with text (as in Wav2Vec’s descendants or image-audio models) and beyond, giving you a sense of where SSL is heading.

**Bonus Exercise:** Try using CLIP for a zero-shot task on your own: for example, find an interesting image and come up with 5 caption prompts that might describe it. Feed them through CLIP and see which caption scores highest. Does it make sense? Alternatively, try the reverse: supply a weird prompt like "a yellow elephant flying" and see which images from a small set CLIP thinks matches best (if none have that, it will choose the closest abstract concept).

## References
- **Radford, A. et al. (2021).** "Learning Transferable Visual Models From Natural Language Supervision." ICML. – The CLIP paper.
- **Jia, C. et al. (2021).** "Scaling Up Visual and Vision-Language Representation Learning With Noisy Text Supervision." ICML. – Google’s ALIGN paper, similar approach as CLIP but with even more data.
- **Mu, J. et al. (2021).** "SLIP: Self-supervision meets Language-Image Pre-training." arXiv. – Combines CLIP’s contrastive loss with SimCLR’s self-supervised loss.
- **Ilharco, G. et al. (2022).** "Patching OpenAI’s CLIP." – Investigates fine-tuning CLIP and proposes ways to do it effectively.
- **Reimers, N. & Gurevych, I. (2019).** "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks." – Not about images, but analogous in using a dual encoder (two BERTs) for sentence pairs with a contrastive-like objective (leads to efficient semantic text similarity). CLIP’s text encoder can be seen as analogous to Sentence-BERT but for cross-modal similarity.
- **OpenAI CLIP model card** – (on the OpenAI or HuggingFace site) discusses limitations and bias of CLIP.
- **Bromley, J. et al. (1993).** "Signature Verification using a Siamese Time Delay Neural Network." – Historically, one of the first uses of dual networks with contrastive loss (Siamese networks), the great-grandparent idea of things like CLIP’s dual encoders.